# 📈 Population Growth Models
## From Exponential Explosions to Carrying Capacity

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-2-population/notebooks/01_population_growth_models.ipynb)

---

> *"The power of population is so superior to the power of the earth to produce subsistence for man, that premature death must in some shape or other visit the human race."* - Thomas Malthus (1798)

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand **exponential growth** and its assumptions
2. Model **logistic growth** with carrying capacity
3. Calculate and interpret **population growth rate (r)**
4. Distinguish between **r-selection** and **K-selection**
5. Apply growth models to **real populations**
6. Predict **doubling time** and **half-life**
7. Understand when each model is appropriate

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import odeint

print("✅ Ready to explore population growth!")
print("📈 Let's model population dynamics!")

---

## 📚 Part 1: Exponential Growth - The J-Curve

### The Simplest Model:

**Exponential growth** assumes:
- ✅ Unlimited resources
- ✅ No predators or disease
- ✅ Constant birth and death rates
- ✅ Ideal conditions

### The Equations:

#### **Differential Form** (rate of change):
```
dN/dt = rN
```

#### **Integral Form** (population size over time):
```
N(t) = N₀ × e^(rt)
```

Where:
- **N** = population size
- **N₀** = initial population size
- **t** = time
- **r** = intrinsic rate of increase (per capita growth rate)
- **e** = Euler's number (≈ 2.718)

### Understanding r (Growth Rate):

**r = b - d**

Where:
- **b** = per capita birth rate
- **d** = per capita death rate

**Interpretation**:
- **r > 0**: Population growing
- **r = 0**: Population stable
- **r < 0**: Population declining

### Doubling Time:

**T_double = ln(2) / r ≈ 0.693 / r**

### The "J-Curve" Shape:

```
           ↑
         /
        /
       /          ← Steep acceleration!
      /
     |
    _|
  ___|
_____|_______________→ Time
```

In [ ]:
# Exponential growth simulation
def exponential_growth(N0, r, t_max):
    """
    Calculate exponential population growth
    """
    t = np.linspace(0, t_max, 1000)
    N = N0 * np.exp(r * t)
    return t, N

# Simulate different growth rates
N0 = 10  # Initial population
t_max = 50  # Time periods

growth_rates = {
    'Fast (r=0.1)': 0.1,
    'Medium (r=0.05)': 0.05,
    'Slow (r=0.02)': 0.02,
    'Declining (r=-0.03)': -0.03
}

fig = go.Figure()

colors = ['red', 'orange', 'blue', 'gray']

for (label, r), color in zip(growth_rates.items(), colors):
    t, N = exponential_growth(N0, r, t_max)
    
    # Calculate doubling time
    if r > 0:
        doubling_time = 0.693 / r
        hover_text = f"{label}<br>Doubling time: {doubling_time:.1f} years"
    elif r < 0:
        half_life = 0.693 / abs(r)
        hover_text = f"{label}<br>Half-life: {half_life:.1f} years"
    else:
        hover_text = f"{label}<br>Stable population"
    
    fig.add_trace(go.Scatter(
        x=t, y=N,
        mode='lines',
        line=dict(width=3, color=color),
        name=hover_text
    ))

fig.update_layout(
    title="📈 Exponential Growth: The J-Curve<br><sub>N(t) = N₀ × e^(rt)</sub>",
    xaxis_title="Time (years)",
    yaxis_title="Population Size (N)",
    height=600,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print("\n📊 Exponential Growth Analysis:\n")
for label, r in growth_rates.items():
    if r > 0:
        doubling_time = 0.693 / r
        final_pop = N0 * np.exp(r * t_max)
        print(f"   {label}:")
        print(f"      • Doubling time: {doubling_time:.1f} years")
        print(f"      • Population after {t_max} years: {final_pop:,.0f}")
        print(f"      • Growth factor: {final_pop/N0:.1f}x\n")
    elif r < 0:
        half_life = 0.693 / abs(r)
        final_pop = N0 * np.exp(r * t_max)
        print(f"   {label}:")
        print(f"      • Half-life: {half_life:.1f} years")
        print(f"      • Population after {t_max} years: {final_pop:.1f}")
        print(f"      • Decline to: {(final_pop/N0)*100:.1f}% of original\n")

print("\n💡 Key Insight:")
print("   Small differences in 'r' lead to HUGE differences over time!")
print("   This is the power of exponential growth.")

---

## 🌍 Part 2: Real Example - Human Population Growth

### Historical Human Growth:

| Year | Population | Doubling Time |
|------|------------|---------------|
| 1 CE | 300 million | ~1500 years |
| 1500 | 500 million | ~300 years |
| 1800 | 1 billion | ~130 years |
| 1930 | 2 billion | ~45 years |
| 1975 | 4 billion | ~40 years |
| 2022 | 8 billion | Still growing |

**Pattern**: Doubling time DECREASING = growth rate INCREASING!

### Current Status:
- **Population**: ~8.1 billion (2024)
- **Growth rate**: ~0.9% per year (declining)
- **Daily increase**: ~200,000 people
- **Projected peak**: ~10 billion (2080s)

In [ ]:
# Real human population data
human_pop_data = pd.DataFrame({
    'Year': [1, 1000, 1500, 1800, 1900, 1950, 1975, 2000, 2025, 2050],
    'Population_billions': [0.3, 0.31, 0.5, 1.0, 1.6, 2.5, 4.0, 6.1, 8.1, 9.7],
    'Growth_rate_%': [0.04, 0.01, 0.1, 0.4, 0.6, 1.8, 1.9, 1.4, 0.9, 0.5]
})

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Human Population Growth', 'Growth Rate Over Time'),
    horizontal_spacing=0.12
)

# Population curve
fig.add_trace(
    go.Scatter(
        x=human_pop_data['Year'],
        y=human_pop_data['Population_billions'],
        mode='lines+markers',
        line=dict(width=3, color='darkgreen'),
        marker=dict(size=10),
        fill='tozeroy',
        fillcolor='rgba(0,128,0,0.2)',
        name='Population'
    ),
    row=1, col=1
)

# Growth rate
fig.add_trace(
    go.Scatter(
        x=human_pop_data['Year'],
        y=human_pop_data['Growth_rate_%'],
        mode='lines+markers',
        line=dict(width=3, color='red'),
        marker=dict(size=10),
        name='Growth Rate'
    ),
    row=1, col=2
)

# Mark key events
events = [
    (1800, 'Industrial Revolution'),
    (1950, 'Post-WWII Baby Boom'),
    (1975, 'Peak Growth Rate')
]

for year, event in events:
    fig.add_vline(
        x=year,
        line_dash="dash",
        line_color="gray",
        annotation_text=event,
        annotation_position="top",
        row=1, col=1
    )

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Population (billions)", row=1, col=1)
fig.update_yaxes(title_text="Growth Rate (%/year)", row=1, col=2)

fig.update_layout(
    title="🌍 Human Population: From Slow Growth to Explosion<br><sub>Classic exponential phase (1800-1975)</sub>",
    height=500,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n🌍 Human Population Milestones:\n")
print("   1 billion:     1804  (first billion)")
print("   2 billion:     1927  (+123 years)")
print("   3 billion:     1960  (+33 years)")
print("   4 billion:     1974  (+14 years)")
print("   5 billion:     1987  (+13 years)")
print("   6 billion:     1999  (+12 years)")
print("   7 billion:     2011  (+12 years)")
print("   8 billion:     2022  (+11 years)")
print("\n📊 Key Observations:")
print("   • Doubling time decreased dramatically 1800-1975")
print("   • Peak growth rate: ~2% in 1968")
print("   • Current growth slowing (demographic transition)")
print("   • Projected to plateau at ~10 billion by 2080")
print("\n💡 Why the slowdown?")
print("   • Falling birth rates (education, contraception)")
print("   • Economic development")
print("   • Urbanization")
print("   • Approaching carrying capacity?")

---

## 🐘 Part 3: Logistic Growth - The S-Curve

### The Realistic Model:

**Problem with exponential growth**: Nothing grows forever!

**Solution**: Add **carrying capacity (K)**

### The Logistic Equation:

#### **Differential Form**:
```
dN/dt = rN(1 - N/K)
```

#### **Integral Form**:
```
N(t) = K / (1 + ((K - N₀)/N₀) × e^(-rt))
```

Where:
- **K** = carrying capacity (maximum sustainable population)
- **(1 - N/K)** = "brake" term (slows growth as N approaches K)

### Understanding the Brake Term:

**When N is small** (N << K):
- (1 - N/K) ≈ 1
- Growth ≈ exponential

**When N = K/2** (halfway to K):
- (1 - N/K) = 0.5
- **Maximum growth rate** (inflection point)

**When N approaches K**:
- (1 - N/K) → 0
- Growth slows to zero

**When N = K**:
- (1 - N/K) = 0
- **No growth** (equilibrium)

### The "S-Curve" Shape:

```
K  _______________  ← Carrying capacity
         /
        /           ← Inflection point (fastest growth)
       /
      /
     |
    _|              ← Exponential phase
_____|____________→ Time
```

In [ ]:
# Logistic growth simulation
def logistic_growth(N0, r, K, t_max):
    """
    Calculate logistic population growth
    """
    t = np.linspace(0, t_max, 1000)
    N = K / (1 + ((K - N0) / N0) * np.exp(-r * t))
    return t, N

# Compare exponential vs logistic
N0 = 10
r = 0.1
K = 1000
t_max = 100

t_exp, N_exp = exponential_growth(N0, r, t_max)
t_log, N_log = logistic_growth(N0, r, K, t_max)

fig = go.Figure()

# Exponential
fig.add_trace(go.Scatter(
    x=t_exp, y=N_exp,
    mode='lines',
    line=dict(width=3, color='red', dash='dash'),
    name='Exponential (unrealistic)'
))

# Logistic
fig.add_trace(go.Scatter(
    x=t_log, y=N_log,
    mode='lines',
    line=dict(width=4, color='blue'),
    name='Logistic (realistic)'
))

# Carrying capacity line
fig.add_hline(
    y=K,
    line_dash="dot",
    line_color="green",
    annotation_text=f"Carrying Capacity (K={K})",
    annotation_position="right"
)

# Mark inflection point (K/2)
inflection_idx = np.argmin(np.abs(N_log - K/2))
t_inflection = t_log[inflection_idx]

fig.add_trace(go.Scatter(
    x=[t_inflection],
    y=[K/2],
    mode='markers',
    marker=dict(size=15, color='orange', symbol='star'),
    name='Maximum growth rate (N=K/2)'
))

fig.update_layout(
    title="📊 Exponential vs Logistic Growth<br><sub>J-curve (red) vs S-curve (blue)</sub>",
    xaxis_title="Time (years)",
    yaxis_title="Population Size (N)",
    height=600,
    template='plotly_white'
)

fig.show()

print("\n📈 Growth Model Comparison:\n")
print("   EXPONENTIAL Growth:")
print("      • Assumes unlimited resources")
print("      • Grows forever (unrealistic)")
print(f"      • Population at year {t_max}: {N_exp[-1]:,.0f}")
print("      • J-shaped curve")
print("\n   LOGISTIC Growth:")
print("      • Recognizes resource limits")
print("      • Levels off at carrying capacity")
print(f"      • Population at year {t_max}: {N_log[-1]:.0f} (≈ K)")
print("      • S-shaped curve")
print(f"\n   ⭐ Inflection Point (fastest growth):")
print(f"      • Occurs at N = K/2 = {K/2}")
print(f"      • Time to reach: {t_inflection:.1f} years")
print("\n💡 Logistic growth is MORE realistic for most populations!")

---

## 🦌 Part 4: Real Example - Deer Population on an Island

### Famous Case: Kaibab Plateau (Arizona)

**Timeline**:
- **1906**: ~4,000 deer, wolves present
- **1906-1924**: Wolves eliminated by humans
- **1924**: ~100,000 deer (25x increase!)
- **1925-1939**: Massive die-off
- **1939**: ~10,000 deer (back to sustainable level)

**What happened?**
1. Removing wolves → exponential growth
2. Exceeded carrying capacity
3. Overgrazed vegetation
4. Mass starvation (overshoot & crash)

**Lesson**: Populations can temporarily EXCEED K, but pay the price!

In [ ]:
# Simulate deer population with overshoot
def deer_population_overshoot():
    """
    Simulate Kaibab Plateau deer population
    """
    years = np.array([1906, 1910, 1915, 1920, 1924, 1926, 1930, 1935, 1939, 1945])
    deer = np.array([4000, 9000, 25000, 65000, 100000, 40000, 20000, 15000, 10000, 10000])
    
    return years, deer

years, deer = deer_population_overshoot()
K_original = 30000  # Estimated original carrying capacity
K_degraded = 10000  # Carrying capacity after habitat degradation

fig = go.Figure()

# Deer population
fig.add_trace(go.Scatter(
    x=years, y=deer,
    mode='lines+markers',
    line=dict(width=4, color='brown'),
    marker=dict(size=12),
    fill='tozeroy',
    fillcolor='rgba(139,69,19,0.3)',
    name='Deer Population'
))

# Carrying capacities
fig.add_hline(
    y=K_original,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Original K ≈ {K_original:,}",
    annotation_position="right"
)

fig.add_hline(
    y=K_degraded,
    line_dash="dot",
    line_color="red",
    annotation_text=f"Degraded K ≈ {K_degraded:,}",
    annotation_position="right"
)

# Mark key events
fig.add_annotation(
    x=1906, y=4000,
    text="Wolves Present<br>Stable Population",
    showarrow=True,
    arrowhead=2,
    ax=-50, ay=-50
)

fig.add_annotation(
    x=1924, y=100000,
    text="OVERSHOOT!<br>Wolves Eliminated",
    showarrow=True,
    arrowhead=2,
    ax=0, ay=-50,
    bgcolor="red",
    font=dict(color="white")
)

fig.add_annotation(
    x=1930, y=20000,
    text="CRASH<br>Mass Starvation",
    showarrow=True,
    arrowhead=2,
    ax=0, ay=50,
    bgcolor="darkred",
    font=dict(color="white")
)

fig.update_layout(
    title="🦌 Kaibab Plateau Deer: Overshoot and Crash<br><sub>Classic example of exceeding carrying capacity</sub>",
    xaxis_title="Year",
    yaxis_title="Deer Population",
    height=600,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n🦌 Kaibab Plateau Deer Case Study:\n")
print("   Phase 1: STABLE (1906)")
print("      • ~4,000 deer")
print("      • Wolves control population")
print("      • Below carrying capacity")
print("\n   Phase 2: EXPONENTIAL GROWTH (1906-1924)")
print("      • Wolves eliminated by hunters")
print("      • Deer population explodes")
print("      • 4,000 → 100,000 (25x in 18 years!)")
print("      • r ≈ 0.18 per year")
print("\n   Phase 3: OVERSHOOT (1924)")
print("      • Peak: ~100,000 deer")
print("      • 3x ABOVE carrying capacity")
print("      • Severe overgrazing")
print("      • Habitat degradation")
print("\n   Phase 4: CRASH (1924-1939)")
print("      • Mass starvation")
print("      • 100,000 → 10,000 (90% die-off)")
print("      • Carrying capacity permanently reduced")
print("\n   Phase 5: NEW EQUILIBRIUM (1939+)")
print("      • ~10,000 deer (stable)")
print("      • Lower K due to habitat damage")
print("\n💡 KEY LESSONS:")
print("   1. Populations CAN exceed K (temporarily)")
print("   2. Overshoot leads to CRASH")
print("   3. Removing predators has consequences")
print("   4. Overgrazing can REDUCE future K")
print("   5. Natural regulation is important!")

---

## 🔄 Part 5: r-Selection vs K-Selection

### Life History Strategies:

Species evolve different strategies based on their environment:

### r-Selected Species ("Live Fast, Die Young")

**Strategy**: Maximize **r** (growth rate)

**Environment**: Unstable, unpredictable, disturbed

**Characteristics**:
- 🐭 **Small body size**
- 📆 **Short lifespan** (days to months)
- 🐣 **Early reproduction** (reproduce quickly)
- 🥚 **Many offspring** (hundreds to millions)
- 👶 **No parental care**
- 📉 **High offspring mortality**
- 🎲 **"Boom and bust" populations**

**Examples**:
- Bacteria
- Insects (flies, aphids)
- Annual plants (weeds)
- Small rodents (mice)
- Many fish species

### K-Selected Species ("Slow and Steady")

**Strategy**: Maximize competitive ability near **K**

**Environment**: Stable, predictable, mature

**Characteristics**:
- 🐘 **Large body size**
- 📅 **Long lifespan** (years to decades)
- 🕰️ **Delayed reproduction**
- 👶 **Few offspring** (1-10)
- 👨‍👩‍👧 **High parental care**
- 📈 **Low offspring mortality**
- ⚖️ **Stable populations near K**

**Examples**:
- Elephants
- Whales
- Trees (oak, sequoia)
- Large predators (bears, eagles)
- Humans

### The Spectrum:

```
r-selected ←――――――――――――――――――→ K-selected

Bacteria                              Elephants
Flies                                 Whales
Weeds          Rabbits      Humans   Trees
```

**Note**: Most species fall somewhere in between!

In [ ]:
# Compare r vs K selected species
rK_comparison = pd.DataFrame({
    'Trait': [
        'Body Size',
        'Lifespan',
        'Age at First Reproduction',
        'Number of Offspring',
        'Offspring Size',
        'Parental Care',
        'Population Stability',
        'Environment',
        'Competition',
        'Growth Rate (r)'
    ],
    'r_Selected': [
        'Small',
        'Short (days-months)',
        'Early',
        'Many (100s-1000s)',
        'Tiny',
        'None',
        'Variable (boom/bust)',
        'Unstable, disturbed',
        'Weak',
        'HIGH'
    ],
    'K_Selected': [
        'Large',
        'Long (years-decades)',
        'Late',
        'Few (1-10)',
        'Large',
        'Extensive',
        'Stable (near K)',
        'Stable, predictable',
        'Strong',
        'LOW'
    ],
    'Examples_r': [
        'Mouse',
        'Fly',
        'Aphid',
        'Salmon',
        'Bacteria',
        'Dandelion',
        'Rabbit',
        'Weeds',
        'Annual plants',
        'Insects'
    ],
    'Examples_K': [
        'Elephant',
        'Human',
        'Whale',
        'Eagle',
        'Oak tree',
        'Gorilla',
        'Bear',
        'Sequoia',
        'Lion',
        'Albatross'
    ]
})

# Create comparison table
fig = go.Figure(data=[go.Table(
    columnwidth=[100, 100, 100, 80, 80],
    header=dict(
        values=['<b>Trait</b>', '<b>r-Selected</b>', '<b>K-Selected</b>', 
                '<b>r-Examples</b>', '<b>K-Examples</b>'],
        fill_color=['lightgray', 'lightcoral', 'lightgreen', 'lightcoral', 'lightgreen'],
        align='left',
        font=dict(size=12)
    ),
    cells=dict(
        values=[rK_comparison[col] for col in rK_comparison.columns],
        fill_color=[['white', 'lightgray'] * 5],
        align='left',
        font=dict(size=11),
        height=35
    )
)])

fig.update_layout(
    title="🔄 r-Selection vs K-Selection Strategies",
    height=600
)

fig.show()

print("\n🔄 Life History Strategies:\n")
print("   🐭 r-SELECTED (fast strategy):")
print("      • Colonize new/disturbed habitats quickly")
print("      • Reproduce before environment changes")
print("      • 'Weedy' species")
print("      • Example: After forest fire, weeds arrive first")
print("\n   🐘 K-SELECTED (slow strategy):")
print("      • Compete well in stable environments")
print("      • Invest heavily in each offspring")
print("      • 'Climax' species")
print("      • Example: Old-growth forest dominants")
print("\n💡 Trade-off:")
print("   Can't be both! Energy goes to either:")
print("   • QUANTITY (many offspring, r-selected)")
print("   • QUALITY (few well-cared offspring, K-selected)")
print("\n🌍 Ecological Succession:")
print("   Disturbed site → r-selected colonize first")
print("                 → Gradually replaced by K-selected")
print("                 → Stable climax community (K-selected dominate)")

---

## 🎓 Summary

### Key Takeaways:

✅ **Exponential growth**: Unlimited resources, J-curve, N(t) = N₀e^(rt)  
✅ **Logistic growth**: Limited resources, S-curve, includes K  
✅ **Growth rate (r)**: r = b - d, determines speed of growth  
✅ **Carrying capacity (K)**: Maximum sustainable population  
✅ **Overshoot & crash**: Populations can exceed K temporarily  
✅ **r-selection**: Fast reproduction, many offspring, unstable environments  
✅ **K-selection**: Slow reproduction, few offspring, stable environments  

### The Growth Models:

| Feature | Exponential | Logistic |
|---------|-------------|----------|
| **Shape** | J-curve | S-curve |
| **Resources** | Unlimited | Limited |
| **Equation** | dN/dt = rN | dN/dt = rN(1-N/K) |
| **Endpoint** | Infinity | Carrying capacity (K) |
| **Realism** | Idealized | More realistic |
| **When applicable** | Early colonization | Most populations |

### When to Use Each Model:

**Exponential**:
- Bacteria in fresh media
- Invasive species arriving
- Initial colonization
- Short-term projections

**Logistic**:
- Established populations
- Resource-limited systems
- Long-term projections
- Conservation planning

### Real-World Applications:

#### 🌍 **Conservation**:
- Estimate minimum viable population
- Predict recovery times
- Set harvest quotas

#### 🎣 **Fisheries**:
- Maximum sustainable yield = r×K/4
- Occurs at N = K/2
- Harvest too much → population crashes

#### 🦟 **Disease Control**:
- Mosquito population management
- Predict outbreak timing
- Evaluate control effectiveness

#### 👥 **Human Population**:
- Demographic projections
- Resource planning
- Carrying capacity debates

### Important Formulas:

**Doubling time**: T_d = 0.693 / r  
**Half-life** (declining): T_h = 0.693 / |r|  
**Time to K/2**: t ≈ (1/r) × ln(K/N₀)  
**Maximum growth rate**: At N = K/2  

### Limitations:

Both models assume:
- Continuous reproduction (not seasonal)
- No age structure
- No time lags
- Constant r and K
- No Allee effects

**Reality is messier!** These are simplified models.

### Next Steps:

In the next notebooks, we'll explore:
- Life tables and survivorship curves
- Age structure effects
- Population regulation mechanisms
- More complex models

---

<div align="center">

**Made with 💚 by The Pattern Hunter Team**

[🏠 Unit 2 Home](../../) | [📓 Next: Life Tables](02_life_tables_survivorship.ipynb)

</div>